# Analysis of Amazon Product Data

After construction of the graph (product-id and link to cross-buys), the following topics will be investigated:
* correlation between in-degrees and salesrank
* and some further smart questions... influence of number of reviews?, are there different sub-graphs like produkt categories?
* 
* 
* 
Data source: https://snap.stanford.edu/data/amazon-meta.html

Data import

In [3]:
# Import graphframes (from Spark-Packages)

from graphframes import *
from pyspark.sql.functions import monotonically_increasing_id, desc


# Get the product data (already pre-processed data in table format)
products = spark.read.csv("/FileStore/tables/ProductData_small.csv", sep=";", header=True)

# The the cross-buy data (similar products)
links = spark.read.csv("/FileStore/tables/LinkData_small.csv", sep=";", header=True)

How many products are we dealing with here?

In [5]:
products.count()

Out[ 2 ]: 10611

quick check of the data formats

In [7]:
display(products.take(3))

Id,ASIN,Group,Salesrank,Reviews,Title
21,0790747324,DVD,795,140,The Time Machine
84,B000063W82,Video,780,14,The Best of Schoolhouse Rock! - 30th Anniversary Edition
252,B0000262WI,Music,581,82,The Köln Concert


And which and how many groups are there?

In [9]:
display(products.select("Group").distinct())

Group
Video
Toy
DVD
Sports
Baby Product
Video Games
Book
Music
Software


In [10]:
products.select("Group").distinct().count()

Out[ 5 ]: 9

And how many products per category?

In [12]:
display(products.groupBy('Group').count().sort('count', ascending=False))

Group,count
Video,2933
Music,2855
Book,2535
DVD,2276
Software,5
Toy,4
Sports,1
Baby Product,1
Video Games,1


In [13]:
display(links.take(3))

src,dest
0790747324,B00007JMD8
0790747324,6305350221
0790747324,B00004RF9B


The graph will be constructed based on the product data. Each product is identified through its unique *ASIN* (Amazon Standard Identification Number) code

GraphFrame library required *ASIN* to be renamed to *id*, but be careful, there is already an "Id" in the DataFrame

In [15]:
# rename "Id" to "Id-nu" to avoid any confusion (nu = not used)
# rename "ASIN" to "id"
ProductVertices = products.withColumnRenamed("Id", "Id-nu").distinct()
ProductVertices = ProductVertices.withColumnRenamed("ASIN", "id").distinct()

# check if done correctly
display(ProductVertices.take(3))

Id-nu,id,Group,Salesrank,Reviews,Title
13260,B000002TIR,Music,2004,16,French Kiss
13683,0140281916,Book,2877,16,Bargaining for Advantage : Negotiation Strategies for Reasonable People
30773,B000001EYO,Music,3579,31,The Big Lebowski: Original Motion Picture Soundtrack


How does the Edge table look like?

In [17]:
display(links.take(1))

src,dest
0790747324,B00007JMD8


And how many edges are there?

In [19]:
links.count()

Out[ 11 ]: 52158

Although not always strictly required, it's good practice to have an ID column for the tables. 

Since the product links does not have one yet, create and additional ID:

In [21]:
# 2. Create Edges
links = links.withColumn("link-id", monotonically_increasing_id())

display(links.take(3))

src,dest,link-id
0790747324,B00007JMD8,0
0790747324,6305350221,1
0790747324,B00004RF9B,2


Salesrank and number of reviews are read as string. Change type to int for calculations.

In [23]:
# change type of features "Salesrank" and "Reviews" to int
ProductVertices = ProductVertices.withColumn('Salesrank', ProductVertices.Salesrank.cast('int'))
ProductVertices = ProductVertices.withColumn('Reviews', ProductVertices.Reviews.cast('int'))
display(ProductVertices.take(1))

Id-nu,id,Group,Salesrank,Reviews,Title
13260,B000002TIR,Music,2004,16,French Kiss


Rename links to similar products ar *src* and *dst*

In [25]:
# rename destination "dest" to "dst"
links = links.withColumnRenamed('dest', 'dst')


Now comes the moment of truth! Let's see if the graph can be constructed!

In [27]:
# Build the GraphFrame based on products and links

ProductGraph = GraphFrame(ProductVertices, links)

# print a sample of the vertices and edges
print("Products: %d" % ProductGraph.vertices.count())
print
print("SAMPLE PRODUCT DATA:")
print(ProductGraph.vertices.take(1))
print

print("Links: %d" % ProductGraph.edges.count())
print
print("SAMPLE LINK DATA:")
print(ProductGraph.edges.take(1))
print

Products: 10611
SAMPLE PRODUCT DATA:
[Row(Id-nu='13260', id='B000002TIR', Group='Music', Salesrank=2004, Reviews=16, Title='French Kiss')]
Links: 52158
SAMPLE LINK DATA:
[Row(src='0790747324', dst='B00007JMD8', link-id=0)]
 Out[ 17 ]: <function print>

The Graph is constructed. Do the fancy analyses!

1) assess the number of in-degress per product and compare it to the salesrank

Plot the next chunk in the following way:

you can look at the distribution of the inDegree-values by clicking on the icon to the right of 'Raw table' and then chosing a histogram, values = 'inDegree' and  30 bins

We can see that most of the inDegree values are between 0 and 2

In [31]:
display(ProductGraph.inDegrees.sort('inDegree', ascending=False))
#inDegRank = ProductGraph.inDegrees.sort('inDegree', ascending=False)

id,inDegree
B00008LDNZ,78
B00005JMAH,55
B00023P4I8,53
B00003CXCT,52
B000634DCW,45
B000068DBC,37
6304765266,36
B00005ATZT,34
B00005JMIJ,33
B00001QEE7,33


In [32]:
maxNumInDeg = ProductGraph.inDegrees.groupBy().max('inDegree')
display(maxNumInDeg)

maxNumInDegValue = str(maxNumInDeg.collect()[0]['max(inDegree)'])
#print(maxNumInDegValue)

maxInDegreeProduct = ProductGraph.inDegrees.filter('inDegree = {}'.format(maxNumInDegValue))
display(maxInDegreeProduct)


id,inDegree
B00008LDNZ,78


In [33]:
# join back to get product info based on max inDegree
ProductMaxInDegree = ProductGraph.vertices.join(maxInDegreeProduct, 'id')
display(ProductMaxInDegree)




id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
B00008LDNZ,548091,DVD,110,83,Laura,78


Plot the following chunk in this way:
Plot this as a bar chart with 'Plot Options':
 - Keys: InDegree
 - Values: Salesrank
 
Unfortunately you can't chose to have a logarithmic y-axis, but you do see that at least for low in-degree values the sales rank is rather bad.

Alternative: scatterplot inDegree vs. salesrank and grouping by product group (Keys=Group)

In [35]:
# repeat exercise from above but with entire list
selection = ProductGraph.inDegrees.sort('inDegree', ascending=False)
#display(selection)

ProductSortInDeg = ProductGraph.vertices.join(selection, 'id').sort('inDegree', ascending=False)
display(ProductSortInDeg)

id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
B00008LDNZ,548091,DVD,110,83,Laura,78
B000068DBC,546689,DVD,109,633,Pulp Fiction (Collector's Edition),37
6304765266,97579,DVD,351,152,While You Were Sleeping,36
B00005ATZT,547040,DVD,1020,140,The Fugitive (Special Edition),34
B00001QEE7,104775,DVD,49,160,The Little Mermaid (Limited Issue),33
6305368171,276371,DVD,236,500,You've Got Mail,33
B00003CXA2,547272,DVD,185,530,Forrest Gump,30
630522577X,548182,DVD,253,180,My Fair Lady,29
B000059HB7,4388,DVD,1877,79,Rio Bravo,29
B000022TSH,544390,DVD,1108,154,Chinatown,27


Plot inDegree vs. Salesrank for Books only
* plot as scatter plot with LOESS smoother

In [37]:
# try to differentiate by product group
ProductSortInDeg = ProductGraph.vertices.join(selection, 'id').sort('inDegree', ascending=False).filter("Group = 'Book'")
display(ProductSortInDeg)

id,Id-nu,Group,Salesrank,Reviews,Title,inDegree
0684801523,199628,Book,956,934,The Great Gatsby,27
0316769487,98756,Book,60,2568,The Catcher in the Rye,24
0446310786,537519,Book,111,1414,To Kill a Mockingbird,20
0694003611,502086,Book,156,339,Goodnight Moon (Board Book),19
0140177396,341570,Book,126,1000,Of Mice and Men (Penguin Great Books of the 20th Century),19
0066620996,154855,Book,29,363,Good to Great: Why Some Companies Make the Leap... and Others Don't,18
0805047905,502784,Book,171,172,"Brown Bear, Brown Bear, What Do You See?",18
0399501487,35512,Book,143,1101,Lord of the Flies,18
0399226907,180888,Book,279,164,The Very Hungry Caterpillar board book,18
0449214923,482263,Book,215,260,Think and Grow Rich,17


Plot the following chunk in this way:
Plot this as a bar chart with 'Plot Options':
 - Keys: OutDegree
 - Values: Salesrank

In [39]:
# repeat exercise from above for outDegree
selection = ProductGraph.outDegrees.sort('outDegree', ascending=True)
#display(selection)

ProductSortOutDeg = ProductGraph.vertices.join(selection, 'id').sort('OutDegree', ascending=True)
display(ProductSortOutDeg)

outDegree is rather boring, since most of the products get assigned 5 products by default that are presumingly similar, as you can see here:

In [41]:
display(ProductSortOutDeg.groupBy("outDegree").count())

plot the output of the next chunk in the following way:
chose a bar chart with options:
  - keys: Reviews
  - values: Salesrank
  
and apply to all rows.
We can see that the less reviews a product has, the less likely it is to be a top selling product (the causal orientation might also be inverse here :) )

In [43]:
# relation of number of reviews and sales rank:
display(ProductGraph.vertices.select("Reviews", "Salesrank").sort("Reviews", ascending = False))


trying to find products which have an edge to another product which in turn has an edge to the first product:

In [45]:
ProductsWithBalancedSimilarity = ProductGraph.find("(v1)-[e1]->(v2); (v2)-[e2]->(v1)")
display(ProductsWithBalancedSimilarity)

how many product-pairs reference each other? can we just calculate it like this?

In [47]:
ProductsWithBalancedSimilarity.count()

does this mean, we have 11202 product-pairs that reference each other? Let's test if our data frame of mutually referencing products lists the pairs once or twice by looking at an example:

In [49]:
display(ProductsWithBalancedSimilarity.filter("(v1.id = '0790747324' and v2.id = 'B00007JMD8') or (v2.id = '0790747324' and v1.id = 'B00007JMD8')"))

So the pairs are listed twice, which kind of makes sense, seeing how we constructed the data frame. This means there are 11202/2 = 5601 product-pairs that reference each other.

Now we want to explore if the "strongly connected components" of our graph are evenly distributed across all groups, which would mean that the probability of a DVD referencing to another DVD is the same as it referencing to e.g. a book, or if the groups tend to stick to themselves.

Here we first calculate the strongly connected components
(small warning: this command takes quite a while, ~1.17 hours, 238 jobs):

In [52]:
connectedComp = ProductGraph.stronglyConnectedComponents(maxIter=3)
#connectedComp.select("id", "component").groupBy("component").count().sort(desc("count")).show()

#ProductGraph.connectedComponents()
display(connectedComp)

The connected components are marked by the same value in the column "component". So if we group the data frame by "component" and "Group" and use the function "count()", we will have as many rows per "component" value as the number of different groups belonging to this cluster.

In [54]:
countGroupComponent= connectedComp.groupBy("component", "Group").count().sort("component")
display(countGroupComponent)

Now how many groups are taking part in each cluster?

In [56]:
countComponentsNumberOfGroups = countGroupComponent.groupBy("component").count().sort(desc("count"))
display(countComponentsNumberOfGroups)

Next we get an overview how many clusters there are with members of *n* groups:

In [58]:
numGroupsCount = countComponentsNumberOfGroups.withColumnRenamed("count", "NumberOfGroupsForComponent").groupBy("NumberOfGroupsForComponent").count()
display(numGroupsCount)

So how many clusters are there in total? Just adding up the values in column "count" in the last chunk results in the number of clusters:

In [60]:
display(numGroupsCount.agg({"count": "sum"}))

6751 is a very high number, seeing that there are just 10611 products in our dataset in total.

And what is the size distribution of the clusters in general (independent which groups its members belong to)?

In [63]:
countConnected = connectedComp.groupBy("component").count().sort(desc("count"))
display(countConnected)

So there is only one really huge cluster of strongly connected components with almost 1000 members. The next biggest one has only a size of 29.

How many of the 6751 clusters have more than 3 members?

In [66]:
countConnected.filter("count > 3").count()